In [19]:
#Version del scraper de Maldita 2.0
#Esta versión ha sido desarrollada para de una forma limpia poder scrapear maldita seleccionando un página en concreto.

In [20]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle
import unicodedata

In [21]:
pagina = 1

In [22]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [23]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(unicodedata.normalize("NFKD", e.get_text()))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Maldita.es"}
    
    return article

In [24]:
lista_enlaces = []
suficiente = True

#Cargo las URLS.
print('Cargando URLs...')

link =  "https://maldita.es/malditobulo/page/" + str(pagina) + "/"

arrayurls= get_urls(link, lista_enlaces)
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/07/02/cientifica-espanola-messi-cristiano-ronaldo-futbolista-coronavirus/', 'https://maldita.es/malditobulo/2020/07/02/cuadro-fotos-sanitarios-fallecidos-covid-19/', 'https://maldita.es/malditobulo/2020/07/02/jorge-javier-vazquez-cocaina-bulo/', 'https://maldita.es/malditobulo/2020/07/02/no-un-magrebi-menor-de-edad-no-recibe-740-euros-los-menores-de-edad-no-pueden-optar-a-ayudas-publicas/', 'https://maldita.es/malditobulo/2020/07/02/sorteo-vale-compra-1000e-mercadona-phishing/', 'https://maldita.es/malditobulo/2020/07/02/playa-algarve-portugal-distanciamiento-social/', 'https://maldita.es/malditobulo/2020/07/02/pablo-iglesias-cocteles-molotov-video-audio-universidad-guardia-civil/', 'https://maldita.es/malditobulo/2020/07/02/senora-mayor-ingresada-hospital-san-juan-de-dios/', 'https://maldita.es/malditobulo/2020/07/02/bulos-desinformaciones-orgullo-lgtbi-2020/', 'https://maldita.es/malditobulo/2020/07/02/pablo-iglesias-besa-tani

In [25]:
lista_articulos

[{'Title': "Cuidado con la foto de una 'científica' que compara el sueldo de los futbolistas con el de un 'biocientífico': es la exministra Isabel García Tejerina y no hay pruebas de que lo haya dicho",
  'Publication_date': '02/07/2020',
  'Content': 'Está circulando una foto de una supuesta científica a la que se le atribuye la siguiente frase: "Ustedes le dan a un futbolista 1 millón de euros por mes y a un biocientífico le dan 1.800 euros por mes. Ahora ustedes buscan el tratamiento para este virus, entonces busquen a Cristiano Ronaldo o a Messi para que encuentren la cura". Es un bulo.La mujer de la foto es la exministra de Agricultura y Pesca, Alimentación y Medio Ambiente Isabel García Tejerina, ingeniera agrónoma, quien ocupó dicha cartera durante el Gobierno de Mariano Rajoy. En concreto, la imagen que se está difundiendo es una captura de esta entrevista que concedió en abril de 2018 en la que no hace mención del coronavirus:Además de que no es una "científica", 

In [26]:
#Guardamos los datos:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Maldita_FN_Pag-' + str(pagina) + '_'+ date
print(filename)

outfile = open(filename,'wb')
pickle.dump(lista_articulos, outfile)  
    
outfile.close()

Maldita_FN_Pag-1_02-07-2020


In [27]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': "Cuidado con la foto de una 'científica' que compara el sueldo de los futbolistas con el de un 'biocientífico': es la exministra Isabel García Tejerina y no hay pruebas de que lo haya dicho",
  'Publication_date': '02/07/2020',
  'Content': 'Está circulando una foto de una supuesta científica a la que se le atribuye la siguiente frase: "Ustedes le dan a un futbolista 1 millón de euros por mes y a un biocientífico le dan 1.800 euros por mes. Ahora ustedes buscan el tratamiento para este virus, entonces busquen a Cristiano Ronaldo o a Messi para que encuentren la cura". Es un bulo.La mujer de la foto es la exministra de Agricultura y Pesca, Alimentación y Medio Ambiente Isabel García Tejerina, ingeniera agrónoma, quien ocupó dicha cartera durante el Gobierno de Mariano Rajoy. En concreto, la imagen que se está difundiendo es una captura de esta entrevista que concedió en abril de 2018 en la que no hace mención del coronavirus:Además de que no es una "científica", 